In [1]:
import torch
from torch import nn


In [28]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] -h+1), (X.shape[1] -w +1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            mul = (X[i:i+h, j:j+w] * K)
#             print(mul)
            Y[i,j] = mul.sum()
    
    return Y

In [30]:
X = torch.tensor([[1.0,2.0,3.0], [2.0,3.0,4.0],[3.0,4.0,5.0]])
K = torch.tensor([[1.0,2.0], [2.0,3.0]])

corr2d(X,K)

tensor([[18., 26.],
        [26., 34.]])

In [34]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [35]:
net = Conv2d(K.shape)

In [36]:
net(X)

tensor([[-3.4153, -5.8228],
        [-5.8228, -8.2303]], grad_fn=<AddBackward0>)

### Edge detection

In [19]:
X = torch.ones((8,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [20]:
K = torch.tensor([[1.0,-1.0]])

In [21]:
corr2d(X,K)
# detects horizontal edges

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [22]:
corr2d(X.t(),K)
# but not vertical edges

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [40]:
X = torch.ones((8,8))

X[:,2:3] = 0
X[:,4:6] = 0

print(X)
Y = corr2d(X,K)
Y, Y.shape
# interesting

tensor([[1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.]])


(tensor([[8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.],
         [8., 3., 5., 3., 0., 5., 8.]]),
 torch.Size([7, 7]))

In [42]:
K.shape

torch.Size([2, 2])

### Learning a convolution

In [63]:
conv2d = nn.Conv2d(1,1,kernel_size=(2,2), bias=False)

X = X.reshape((1,1,8,8))
Y = Y.reshape((1,1,7,7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    
    if (i+1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum()}')

batch 2, loss 20305.61328125
batch 4, loss 5402875.0
batch 6, loss 1439295744.0
batch 8, loss 383420596224.0
batch 10, loss 102141192568832.0
